# Scraping using Geocoding API of OpenStreet Maps(OSM)<br>
We will find address using web scrapping and using python

In [50]:
# import nominatim api
from geopy.geocoders import Nominatim

In [51]:
# activate nominatim geocoder
locator = Nominatim(user_agent = "http")
# type any address
location = locator.geocode("Sree Padmanabhaswamy Temple")

In [52]:
# get the latitide and longitude of the place
print(f'Address : {location.address}')
print(f'Latitude : {location.latitude}\nLongitude : {location.longitude}')

Address : Sree Padmanabhaswamy Temple, Padmavilasam Road, Padmanagar, Fort, Thiruvananthapuram, Kerala, 695001, India
Latitude : 8.482769950000002
Longitude : 76.94370162822179


In [53]:
# print all the information about ebtered address thtat the API has to offer
location.raw

{'boundingbox': ['8.482634', '8.4828919', '76.9434819', '76.9439228'],
 'class': 'amenity',
 'display_name': 'Sree Padmanabhaswamy Temple, Padmavilasam Road, Padmanagar, Fort, Thiruvananthapuram, Kerala, 695001, India',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//place_of_worship_unknown3.p.20.png',
 'importance': 0.30100000000000005,
 'lat': '8.482769950000002',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '76.94370162822179',
 'osm_id': 480106233,
 'osm_type': 'way',
 'place_id': 207363957,
 'type': 'place_of_worship'}

In [54]:
# location 2
my_college = locator.geocode("COllege Of Engineering Pune")
# get the latitide and longitude of the place
print(f'Address : {my_college.address}')
print(f'Latitude : {my_college.latitude}\nLongitude : {my_college.longitude}')

Address : College of Engineering, Pune, Old Pune–Mumbai Highway, Shivajinagar, Pune City, Pune District, Maharashtra, 411001, India
Latitude : 18.52930005
Longitude : 73.85682537025511


In [55]:
my_college.raw

{'boundingbox': ['18.5285846', '18.5315557', '73.8545636', '73.8586755'],
 'class': 'amenity',
 'display_name': 'College of Engineering, Pune, Old Pune–Mumbai Highway, Shivajinagar, Pune City, Pune District, Maharashtra, 411001, India',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//education_school.p.20.png',
 'importance': 0.7813885782638657,
 'lat': '18.52930005',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '73.85682537025511',
 'osm_id': 3718263,
 'osm_type': 'relation',
 'place_id': 283307234,
 'type': 'college'}

# Get the data - BBC Weather location service<br>
We will scrape the location ID of any place that we enter.

In [56]:
import os

import requests # get request from webpage
import json # convert API to json format

from urllib.parse import urlencode
import numpy as np
import pandas as pd
import re
from datetime import datetime

from bs4 import BeautifulSoup as bs

In [57]:
# Enter city name and convert url into compact form
city_name = "Pune"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    's': city_name.lower(),
    'a': 'true',
    'format': 'json'})
location_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&s=pune&a=true&format=json'

In [58]:
# convert url into json format
result_json = requests.get(location_url).json()
result_json

{'response': {'results': {'results': [{'container': 'India',
     'containerId': 1269750,
     'country': 'IN',
     'id': '1259229',
     'language': 'en',
     'latitude': 18.51957,
     'longitude': 73.85535,
     'name': 'Pune',
     'placeType': 'settlement',
     'timezone': 'Asia/Kolkata'},
    {'container': 'India',
     'containerId': 1269750,
     'country': 'IN',
     'id': '6452628',
     'language': 'en',
     'latitude': 18.58223,
     'longitude': 73.91988,
     'name': 'Pune Airport',
     'placeType': 'airport',
     'timezone': 'Asia/Kolkata'}],
   'totalResults': 2}}}

In [59]:
# we will get id from the results dictionary
result_json['response']['results']['results'][0]['id']

'1259229'

In [60]:
# create a function to get the id
def get_id(city):
    city = city.lower()
    location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    's': city,
    'a': 'true',
    'format': 'json'})
    city_url_json = requests.get(location_url).json()
    city_id = city_url_json['response']['results']['results'][0]['id']
    return city_id
city = "New Delhi"
city_id = get_id(city)
print(city_id)

1261481


In [61]:
# get the url from which we are going to scrape data

url = 'https://www.bbc.com/weather/' + city_id
#print(url)
response = requests.get(url)

Initiate Beautiful Soup

In [62]:
soup = bs(response.content, 'html.parser')
# we can get these class by inspecting the site
daily_high_values = soup.find_all(class_ = 'wr-day-temperature__high-value')
daily_low_values =  soup.find_all(class_ = 'wr-day-temperature__low-value')

# get gaily high temperature values
daily_high = []
for row in daily_high_values:
    high = row.span.text
    daily_high.append(high)
print(f'Daily High temperature is: {daily_high}')

# get daily low temperature values
daily_low = []
for row in daily_low_values:
    low = row.span.text
    daily_low.append(low)
print(f'Daily Low temperature is: {daily_low}')

Daily High temperature is: ['17° 63°', '19° 65°', '18° 65°', '20° 68°', '21° 70°', '22° 71°', '21° 70°', '19° 67°', '19° 66°', '19° 66°', '19° 65°', '19° 66°', '19° 67°', '20° 67°']
Daily Low temperature is: ['7° 44°', '6° 43°', '7° 44°', '8° 47°', '9° 48°', '9° 49°', '11° 51°', '9° 49°', '8° 46°', '7° 44°', '6° 43°', '6° 43°', '6° 43°', '6° 43°']


In [63]:
daily_high[0].strip().split()[0]

'17°'

In [64]:
daily_high = [i.strip().split()[0] for i in daily_high]
print(f'Daily High temperature(°C) is: {daily_high}')

daily_low = [i.strip().split()[0] for i in daily_low]
print(f'Daily Low temperature(°C) is: {daily_low}')

Daily High temperature(°C) is: ['17°', '19°', '18°', '20°', '21°', '22°', '21°', '19°', '19°', '19°', '19°', '19°', '19°', '20°']
Daily Low temperature(°C) is: ['7°', '6°', '7°', '8°', '9°', '9°', '11°', '9°', '8°', '7°', '6°', '6°', '6°', '6°']


In [65]:
# find summary
daily_summary = soup.find('div', attrs={'class' : 'wr-day-summary'})
daily_summary.text

'Sunny and light windsSunny intervals and light windsSunny intervals and light windsSunny and a gentle breezeSunny intervals and light windsSunny intervals and light windsLight rain showers and light windsLight rain showers and light windsLight cloud and light windsSunny intervals and light windsSunny and light windsSunny and light windsSunny and light windsSunny and light winds'

In [66]:
# # print(type(summary))
# temp = ""
# # for i in range(summary):
# #     if 

# Scrapping IMDB Data<br>
We will represent 250 list of movies in a tabular form using python.<br> We will use BeautifulSoup library to scrape data from web.


In [89]:
from bs4 import BeautifulSoup as bs

In [90]:
req = requests.get("https://www.imdb.com/chart/top/")

# convert to a beautiful soup object
soup = bs(req.content)

# Print out HTML
# adds indentation to the data and created it into standard HTML format
contents = soup.prettify() 

In [91]:
movie_title = []
movie_year = []
movie_rating = []

Extract HTML Tag contents

In [92]:
# The movie name and year are nested under the parent class "chart full_width"
imdb_table = soup.find(class_ = "chart full-width")

In [93]:
# tag for name of movie
movie_title_column = imdb_table.find_all(class_ = "titleColumn")
#print(movie_title_column)

In [94]:
# tag for IMDB rating of movie
movie_rating_columns = imdb_table.find_all(class_ = "ratingColumn imdbRating")

In [95]:
# Extract the titles of movies from movie_title_column
for row in movie_title_column:
    title = row.a.text # extract the text from tag "a"
    movie_title.append(title)
#movie_title = list(movie_title)
len(movie_title)

250

In [96]:
movie_title[:10] # Movie

['The Shawshank Redemption',
 'The Godfather',
 'The Godfather: Part II',
 'The Dark Knight',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'The Good, the Bad and the Ugly',
 'The Lord of the Rings: The Fellowship of the Ring']

In [97]:
# Extract the year of the movir release
for row in movie_title_column:
    year = row.span.text # tag for movie year
    movie_year.append(year)

In [98]:
movie_year[:10] # Year

['(1994)',
 '(1972)',
 '(1974)',
 '(2008)',
 '(1957)',
 '(1993)',
 '(2003)',
 '(1994)',
 '(1966)',
 '(2001)']

In [99]:
# Extract the imbd rating
for row in movie_rating_columns:
    rating = row.strong.text # "strong" is tag for rating
    movie_rating.append(rating)

In [100]:
movie_rating[:10] # IMDB

['9.2', '9.1', '9.0', '9.0', '8.9', '8.9', '8.9', '8.8', '8.8', '8.8']

Merge all in a single data frame

In [101]:
movie_df = pd.DataFrame({"Movie Name" : movie_title, "Year of Release" : movie_year, "IMDB Rating" : movie_rating})
movie_df.head(10)

,Movie Name,Year of Release,IMDB Rating
0,The Shawshank Redemption,(1994),9.2
1,The Godfather,(1972),9.1
2,The Godfather: Part II,(1974),9.0
3,The Dark Knight,(2008),9.0
4,12 Angry Men,(1957),8.9
5,Schindler's List,(1993),8.9
6,The Lord of the Rings: The Return of the King,(2003),8.9
7,Pulp Fiction,(1994),8.8
8,"The Good, the Bad and the Ugly",(1966),8.8
9,The Lord of the Rings: The Fellowship of the Ring,(2001),8.8


# Scrapping Wikipedia

In [102]:
# import wikipedia library
!pip install wikipedia
import wikipedia as wk

In [103]:
# search any keyword
print(wk.search("COEP"))

['College of Engineering, Pune', 'Sindhutai Sapkal', 'Anil Sahasrabudhe', 'Maharashtra', 'List of Indian engineering colleges before Independence', 'Prohibition in the United States', 'Community organizing', 'List of Maharashtra Government Engineering Colleges', 'Swayam', 'Thermo Fisher Scientific']


In [104]:
print(wk.search("COEP", results = 3))

['College of Engineering, Pune', 'Sindhutai Sapkal', 'Anil Sahasrabudhe']


In [105]:
print(wk.summary("College Of Engineering Pune"))

College of Engineering Pune (COEP), is an autonomous institute of the Government of Maharashtra, situated in Pune, Maharashtra, India. Established in 1854 it is the 3rd oldest engineering college in India, after IIT Roorkee (1847) and College of Engineering, Guindy, Chennai (1859). The students and alumni of College of Engineering, Pune are colloquially referred to as COEPians. It was the first technical institution in India to offer mechanical engineering.


In [106]:
print(wk.summary("College Of Engineering Pune", sentences = 2))

College of Engineering Pune (COEP), is an autonomous institute of the Government of Maharashtra, situated in Pune, Maharashtra, India. Established in 1854 it is the 3rd oldest engineering college in India, after IIT Roorkee (1847) and College of Engineering, Guindy, Chennai (1859).


In [107]:
# to get complete page
coep_page = wk.page("College Of Engineering Pune")
print(coep_page.content)

College of Engineering Pune (COEP), is an autonomous institute of the Government of Maharashtra, situated in Pune, Maharashtra, India. Established in 1854 it is the 3rd oldest engineering college in India, after IIT Roorkee (1847) and College of Engineering, Guindy, Chennai (1859). The students and alumni of College of Engineering, Pune are colloquially referred to as COEPians. It was the first technical institution in India to offer mechanical engineering.


== History ==

The college was one of the first few colleges established by the British to serve the Indian subcontinent. The foundation stone of the new college was laid by the Governor of Bombay, Sir Bartle Frere, who was to become the Vice-Chancellor of the University, on 5 August 1865. In June of the following year, Theodore Cooke M.A., was appointed the Principal. He went on to hold the office for next 28 years.

Initially, it was started on 16 December 1854, to train public works department (PWD) officials and was housed in 

In [108]:
# get url of page
print(coep_page.url)

https://en.wikipedia.org/wiki/College_of_Engineering,_Pune


In [109]:
# get title of page
print(coep_page.title)

College of Engineering, Pune


In [110]:
# get images of keyword
print(coep_page.images)

['https://upload.wikimedia.org/wikipedia/commons/2/27/Bust_of_Sir_Visvesvarayya_facing_the_College%27s_Main_building.jpg', 'https://upload.wikimedia.org/wikipedia/commons/a/a7/COEP_Date.JPG', 'https://upload.wikimedia.org/wikipedia/commons/4/4e/COEP_Main_building.JPG', 'https://upload.wikimedia.org/wikipedia/commons/c/ca/College_of_Engineering%2C_Pune_Auditorium_%28Stage_View%29.jpg', 'https://upload.wikimedia.org/wikipedia/commons/c/ca/People_at_83rd_Regatta.jpg', 'https://upload.wikimedia.org/wikipedia/commons/6/63/Pune_ShaniwarWada_DelhiGate.jpg', 'https://upload.wikimedia.org/wikipedia/commons/0/04/The_Mind-Spark_Champion%27s_Trophy.jpg', 'https://upload.wikimedia.org/wikipedia/en/f/f0/College_of_Engineering%2C_Pune_logo.jpg', 'https://upload.wikimedia.org/wikipedia/en/4/41/Flag_of_India.svg']


In [111]:
# the tables on the page will be stored as list
# and can be converttd by using pandas

coep_html = wk.page("College Of Engineering Pune").html().encode("UTF-8")
coep_html
import pandas as pd
coep_df = pd.read_html(coep_html)[4]
coep_df

,0,1
0,Palaces and forts,Shaniwar Wada Vishrambaug Wada Shinde Chhatri ...
1,Temples,Puneshwar Narayaneshwar Pataleshwar Kasba Gana...
2,Museums,Raja Dinkar Kelkar Museum Mahatma Phule Museum...
3,Parks,Bund Garden Rajiv Gandhi Zoological Park Shuny...
4,Others,Bal Gandharva Ranga Mandir Tilak Smarak Ranga ...


# Basic Pandas

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
excel_file = '/content/drive/MyDrive/Colab Notebooks/Diploma/Data Sets/IFCB2009_85.xlsx'
excel_file_df = pd.read_excel(excel_file)
excel_file_df.head()

,BANK,IFSC,BRANCH,ADDRESS,CITY 1,CITY2,STATE,PHONE
0,LAXMI VILAS BANK,LAVB0000101,CHENNAI ZO,"189,IST FLOOR,AARTHI CHAMBERS, ANNASALAI,CHENN...",CHENNAI,CHENNAI,TAMIL NADU,0
1,LAXMI VILAS BANK,LAVB0000102,ACCOUNTS DEPT,"ADMINISTRATIVE OFFICE,NO2,SALEM MAIN ROAD,KATH...",KARUR,KARUR,TAMIL NADU,0
2,LAXMI VILAS BANK,LAVB0000103,ADONI,12 BY 510 MUNICIPAL MAIN ROAD P B NO 17ADONI 5...,KURNOOL,ADONI,ANDHRA PRADESH,252427
3,LAXMI VILAS BANK,LAVB0000104,AMBUR,"NO 87, BROAD BAZAARP B NO 35 AMBUR635802 VELLORE",VELLORE,AMBUR,TAMIL NADU,242126
4,LAXMI VILAS BANK,LAVB0000105,ANBIL,"2 BY 237,MAIN ROAD MANGAMMALPURAM,ANBIL 621 70...",TIRUCHIRAPALLI,LALGUDI,TAMIL NADU,2541287


In [114]:
excel_file_df.shape

(601, 8)

In [115]:
excel_file_df['CITY 1'].value_counts()

CHENNAI            41
COIMBATORE         29
SALEM              22
ERODE              21
KARUR              21
                   ..
MANCHERIAL          1
BAGALKOT            1
JAYASHANKAR         1
PURBI SINGHBHUM     1
PONDICHERRY         1
Name: CITY 1, Length: 173, dtype: int64

In [116]:
locator = Nominatim(user_agent="http")
location = locator.geocode('CHENNAI')
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 13.0836939, Longitude = 80.270186


# Scraping PDFs

In [ ]:
import os
!pip install tabula-py
import tabula
import urllib.request
from urllib.parse import urljoin
url = 'https://www.premierleague.com/publications'

# Specify the folder for the files to be downloaded
folder_loc = r'/content/drive/MyDrive/Colab Notebooks/Diploma/Tools in Data Science/Premier_League'

# if not os.path.exists(folder_loc):
#     os.mkdir(folder_loc)

Actual Code

In [ ]:
response = requests.get(url)
soup = bs(response.text, 'html.parser')

# extract all tags that end in .pdf
for link in soup.select('a[href$=".pdf"]'):
    # Name of the file to be downloaded
    filename = os.path.join(folder_loc, link['href'].split('/')[-1])
    print(filename)
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url, link['href'])).content)

Read table from file and export it to PDF

In [49]:
import tabula
df = tabula.read_pdf(folder_loc + '/This-is-PL-Interactive-Combined.pdf', pages='18')
df

[                                           Unnamed: 0  ... Total payment
 0                                                 NaN  ...       £149.4m
 1                                                 NaN  ...       £149.8m
 2                                                 NaN  ...       £144.4m
 3                                                 NaN  ...       £145.9m
 4                                                 NaN  ...       £141.7m
 5                                                 NaN  ...       £142.0m
 6                                                 NaN  ...       £119.8m
 7                                                 NaN  ...       £128.0m
 8                                                 NaN  ...       £118.2m
 9                                               £100m  ...       £123.0m
 10                                                NaN  ...           NaN
 11                                                NaN  ...           NaN
 12                                   